# how to speedup the Submission Process for Neural Networks written in Tensorflow/Keras

To not get a Timeout at Submission Time you need around 50 it/sec in the submission loop. Manny Complex Models have problems with this Time constraint since the API only lets us predict in single mode instead of in Batch Mode. At least for Tensorflow/Keras I have found a weird behaviour, which takes up a lot of time and so causes the Submission to fail due to a timeout.

For demonstration I will just build a simple Keras model and use it imediately for predictions. Since the Model doesn't get trained at all the Predictions are going to be way off, but this is for demonstration Purpouses only!

In [ ]:
import numpy as np
import tensorflow.keras as keras
import janestreet
from tqdm import tqdm
import time

In [ ]:
def build_model(input_dim):
    i = keras.Input(input_dim)
    
    x = keras.layers.Dense(64, activation="relu")(i)
    x = keras.layers.BatchNormalization()(x)
    
    x = keras.layers.Dense(32, activation="relu")(x)
    x = keras.layers.BatchNormalization()(x)
    
    x = keras.layers.Dense(1, activation="sigmoid")(x)
    
    model = keras.Model(i, x)
    model.compile(optimizer="Adam", loss="MSE")
    
    return model

In [ ]:
model = build_model(130)

In [ ]:
features = ['feature_0',
 'feature_1',
 'feature_2',
 'feature_3',
 'feature_4',
 'feature_5',
 'feature_6',
 'feature_7',
 'feature_8',
 'feature_9',
 'feature_10',
 'feature_11',
 'feature_12',
 'feature_13',
 'feature_14',
 'feature_15',
 'feature_16',
 'feature_17',
 'feature_18',
 'feature_19',
 'feature_20',
 'feature_21',
 'feature_22',
 'feature_23',
 'feature_24',
 'feature_25',
 'feature_26',
 'feature_27',
 'feature_28',
 'feature_29',
 'feature_30',
 'feature_31',
 'feature_32',
 'feature_33',
 'feature_34',
 'feature_35',
 'feature_36',
 'feature_37',
 'feature_38',
 'feature_39',
 'feature_40',
 'feature_41',
 'feature_42',
 'feature_43',
 'feature_44',
 'feature_45',
 'feature_46',
 'feature_47',
 'feature_48',
 'feature_49',
 'feature_50',
 'feature_51',
 'feature_52',
 'feature_53',
 'feature_54',
 'feature_55',
 'feature_56',
 'feature_57',
 'feature_58',
 'feature_59',
 'feature_60',
 'feature_61',
 'feature_62',
 'feature_63',
 'feature_64',
 'feature_65',
 'feature_66',
 'feature_67',
 'feature_68',
 'feature_69',
 'feature_70',
 'feature_71',
 'feature_72',
 'feature_73',
 'feature_74',
 'feature_75',
 'feature_76',
 'feature_77',
 'feature_78',
 'feature_79',
 'feature_80',
 'feature_81',
 'feature_82',
 'feature_83',
 'feature_84',
 'feature_85',
 'feature_86',
 'feature_87',
 'feature_88',
 'feature_89',
 'feature_90',
 'feature_91',
 'feature_92',
 'feature_93',
 'feature_94',
 'feature_95',
 'feature_96',
 'feature_97',
 'feature_98',
 'feature_99',
 'feature_100',
 'feature_101',
 'feature_102',
 'feature_103',
 'feature_104',
 'feature_105',
 'feature_106',
 'feature_107',
 'feature_108',
 'feature_109',
 'feature_110',
 'feature_111',
 'feature_112',
 'feature_113',
 'feature_114',
 'feature_115',
 'feature_116',
 'feature_117',
 'feature_118',
 'feature_119',
 'feature_120',
 'feature_121',
 'feature_122',
 'feature_123',
 'feature_124',
 'feature_125',
 'feature_126',
 'feature_127',
 'feature_128',
 'feature_129']

## Simple prediction Loop

In [ ]:
janestreet.competition.make_env.__called__ = False
env = janestreet.make_env()

start_time = time.time()
for (test_df, pred_df) in tqdm(env.iter_test()):
    x_tt = test_df.loc[:, features].values
    if np.isnan(x_tt[:, 1:].sum()):  # simply ignoring missing values and imediately predicting 0
        pred_df.action = 0
    else:
        pred = model.predict(x_tt)
        pred_df.action = np.where(pred > 0.5, 1, 0).astype(int)
    env.predict(pred_df)
print(f"took: {time.time() - start_time} seconds")

As we can see this simple Neural Network is to slow for submitting. This Code would result in an Timeout on Submission. I also analyzed the Code and found out that as expectet most of the time went into producing the Neural Network Predictions. As you can see we used the ***model.predict*** Function to get our Predictions.

In [ ]:
janestreet.competition.make_env.__called__ = False
env = janestreet.make_env()

start_time = time.time()
for (test_df, pred_df) in tqdm(env.iter_test()):
    x_tt = test_df.loc[:, features].values
    if np.isnan(x_tt[:, 1:].sum()):  # simply ignoring missing values and imediately predicting 0
        pred_df.action = 0
    else:
        pred = model(x_tt, training=False)
        pred_df.action = np.where(pred > 0.5, 1, 0).astype(int)
    env.predict(pred_df)
print(f"took: {time.time() - start_time} seconds")

 this time we changed the ***model.predict()*** function to simply calling the model itselv with the ***model()*** function. As you can see the Prediction Loop is now way faster! For me this is quite a weird behaviour since both methods results in exactly the same predictions but the direct ***model()*** function just is way faster. I also tried this in other real Notebooks and the direct ***model()*** function made the difference for all these Notebooks, before I used the ***model.predict()*** function and even the most simple models got an timeout on submission. Now I'm able to train quite large and complex Models wihout beeing in troubble for an timeout on Prediction.

### In one Line: use ***model()*** instead of ***model.predict()*** !!!